In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 # Collection of library and modules that are going to be used throughout the project



In [ ]:
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,KFold,GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,StackingRegressor,BaggingRegressor, VotingRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.pipeline import Pipeline


# Reading of the dataset from the training of various models

* -- identifying columns and datatypes

* -- removes some of the columns based on their insignificance to our model dependent variable

* -- Identify categorical and numerical data and encode them if necessary

* -- Scaling of data to avoid biases

In [ ]:
tdata = pd.read_csv('drive/MyDrive/IntroAi/players_21.csv')


In [ ]:
# checking the number of colums and their datatypes
tdata.info()

In [ ]:
tdata.columns


In [ ]:
#checking our target variable and see if there is some missing values
tdata['overall']

In [ ]:
# copy the dataset into a new variable to ensure that if we make some changes we can still go back to the original dataset
newdata = tdata.copy()

In [ ]:
newdata.head()

# Cleaning of the data

* Dropping all columns with more than 30% Nan
* Drop other non numerical columns that by definition of the dataset will not contribute to our dependent variable

In [ ]:
# Finding of the percentage of missing value in each column
nan_percent = (newdata.isna().sum()/len(newdata))*100

In [ ]:
# Dropping of the same of the columns with >= 30% NaN
columns_to_drop = nan_percent[nan_percent>=30].index

In [ ]:
# Checking the columns identified
columns_to_drop

In [ ]:
# Dropping of the columns identified above
newdata = newdata.drop(columns = columns_to_drop, axis = 1)

# Separation of numerical and categorical data

* identify the categorical and put them in one subset dataframe
* identify the numerical data and put them in one subset dataframe
* encode the categorical data to make them numeric
* Concatinate the the two dataset to form the final dataset

In [ ]:
# numerical data
numedata = newdata.select_dtypes(exclude=['object'])

In [ ]:
# categical data
catedata = newdata.select_dtypes(include =['object'])

In [ ]:
# Inspecting the numerical data
numedata.info()

In [ ]:
# inspecting the categorical data
catedata.info()

transformation of categorical data into numerical data

In [ ]:
columns = catedata.columns

In [ ]:
columns

In [ ]:
# filling numerical missing value with mean
numedata = numedata.fillna(numedata.mean())

In [ ]:
# factorization of the categorical data

factoData = pd.DataFrame()
for col in columns:
    factoData[col] = pd.factorize(catedata[col])[0]



In [ ]:
# concatenation of the categorical and numerical data
data = pd.concat([numedata,factoData], axis =1)

In [ ]:
data = data.fillna(data.mean())

In [ ]:
data = data.drop(["player_url", "short_name","long_name", "player_face_url"], axis = 1)

In [ ]:
y= data["overall"]

In [ ]:
x = data.drop(["overall"], axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled = StandardScaler()

In [ ]:
scaleddata = scaled.fit_transform(x)

In [ ]:
x = pd.DataFrame(scaleddata, columns = x.columns)

In [ ]:
x.info()

#Different Regression model to use for feature importances

1st model

In [ ]:
randommodel = RandomForestRegressor(n_estimators=150, random_state=42)
randommodel.fit(x,y)

In [ ]:
feature_importances = randommodel.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': x.columns, 'Importance': feature_importances})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
# looking at the best 10 columns that contribute highly on the predicted model
feature_importance_df[0:10]

2nd model

In [ ]:
tree = DecisionTreeRegressor()

In [ ]:
tree.fit(x,y)

In [ ]:
feature_importances = tree.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance = pd.DataFrame({'Feature': x.columns, 'Importance': feature_importances})

# Sort features by importance in descending order
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)


In [ ]:
feature_importance[0:10]

#Ensemble learning with the best five columns of the the dataset

* Training of at least 3 models
* Fine tune them

In [ ]:
columns = ["value_eur","release_clause_eur", "age" ,"potential" ,"movement_reactions", "overall"]

* dataset that we are going to use for the model training

In [ ]:
finaldata = data[columns]

In [ ]:
finaldata.head()

* Splitting of data into dependent and independent variables

In [ ]:
Yfinal = finaldata["overall"]

In [ ]:
Xfinal = finaldata.drop(["overall"], axis =1)

* Scaling of independent variables

In [ ]:
# scaling of Xfinal data

scal = StandardScaler()
scaledfinal = scal.fit_transform(Xfinal)

Xfinal = pd.DataFrame(scaledfinal, columns = Xfinal.columns)


* first model

In [ ]:
svr_model = SVR(kernel = "rbf")
svr_model.fit(Xfinal,Yfinal)

* Second  model

In [ ]:
tree_model = DecisionTreeRegressor()
tree_model.fit(Xfinal, Yfinal)

* 3rd model

In [ ]:
random_model = RandomForestRegressor(n_estimators = 150)
random_model.fit(Xfinal,Yfinal)

* 4th model

In [ ]:
gradient_model = GradientBoostingRegressor(n_estimators = 1000, learning_rate=0.2)
gradient_model.fit(Xfinal,Yfinal)

* 5th model

In [ ]:
nu_model = NuSVR(kernel = "rbf", nu = 0.9)

nu_model.fit(Xfinal, Yfinal)

* 6th model

In [ ]:
base_models = [("decision_tree",DecisionTreeRegressor()),
               ("SVR", SVR(kernel = "rbf")),
               ("random_forest", RandomForestRegressor(n_estimators = 150, random_state =42)),
               ('Gradient_model',GradientBoostingRegressor(n_estimators = 150, learning_rate = 0.1, max_depth =1, random_state =42)),
               ("Nu_svr", NuSVR(kernel ="rbf",nu = 0.5))
               ]

* 7th model

In [ ]:
stacking_model = StackingRegressor(estimators=base_models, final_estimator= LinearRegression())

In [ ]:
stacking_model.fit(Xfinal,Yfinal)

* 8th model

In [ ]:
voting_model = VotingRegressor(estimators = [("decision_tree",DecisionTreeRegressor()),
               ("SVR", SVR(kernel = "rbf")),
               ("random_forest", RandomForestRegressor(n_estimators = 150, random_state =42)),
               ('Gradient_model',GradientBoostingRegressor(n_estimators = 150, learning_rate = 0.1, max_depth =1, random_state =42)),
               ])

In [ ]:
voting_model.fit(Xfinal,Yfinal)

* 9th model

In [ ]:
bagging_model = BaggingRegressor(base_estimator = SVR(kernel = "rbf"), n_estimators =50, random_state= 42)

In [ ]:
bagging_model.fit(Xfinal,Yfinal)

#cross-validation to find a better parameters for our best model

In [ ]:
estimators = [
    ("SVR", SVR(kernel="rbf")),
    ("Gradient_model", GradientBoostingRegressor())
]

param_grid = {
    'SVR__C': [1, 10, 100],
    'Gradient_model__n_estimators': [50, 100, 150],
    'Gradient_model__learning_rate': [0.01, 0.1, 0.5]
}


voting_regressor = VotingRegressor(estimators=estimators)

# Perform grid search with cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(voting_regressor, param_grid, cv=kfold, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(Xfinal, Yfinal)

# Print the best hyperparameters and corresponding RMSE score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best RMSE Score: ", np.sqrt(-grid_search.best_score_))

In [ ]:

random_seed = 42
estimators = [
    ("SVR", SVR(kernel="rbf")),
    ("Gradient_model", GradientBoostingRegressor())
]

param_grid = {
    'SVR__C': [1, 10, 100],
    'Gradient_model__n_estimators': [50, 100, 150],
    'Gradient_model__learning_rate': [0.01, 0.1, 0.5]
}

voting_regressor = VotingRegressor(estimators=estimators)
pipeline = Pipeline([("estimators", voting_regressor)])




In [ ]:
# Fit the model and print the results
grid_search.fit(Xfinal, Yfinal)
best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)

print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

print(f"Best RMSE Score: {best_score}")


In [ ]:
# Trying the final model with the best paramaters



In [ ]:
voting_final = VotingRegressor(estimators=[
               ("SVR", SVR(kernel='rbf', C=100, gamma=0.1)),
               ('Gradient_model',GradientBoostingRegressor(n_estimators = 150, learning_rate = 0.5, max_depth =10, random_state =42)),
               ])

In [ ]:
voting_final.fit(Xfinal,Yfinal)

#Time to test our models on unseen data:

* clean the testing dataset
* Scaling the them
* splitting the data into dependent and independent variables
* No need for separation of categorical and numerical because we found using feature importance method that all columns that are contributing highly on dependent variable are numeric
* Use cross validation to check the best parameters for our ensemble learning models
* select the best models based on it MAE and RMSE

In [ ]:
# importing testing dataset
testdata =pd.read_csv('drive/MyDrive/IntroAi/players_22.csv', usecols= columns)

In [ ]:
# inspecting for messing values in each columms
testdata.info()

* Cleaning and scaling of our testing dataset

In [ ]:
testdata = testdata.dropna()

In [ ]:
testdata = testdata[columns]

In [ ]:
testdata.info()

* Splitting our dataset into independent and independent variables

In [ ]:
Ytestdata = testdata['overall']

In [ ]:
Xtestdata = testdata.drop(["overall"], axis =1)

In [ ]:
scalerdata = scal.transform(Xtestdata)

In [ ]:
scalerdata

In [ ]:
finalxdata = pd.DataFrame(scalerdata, columns =Xtestdata.columns)

In [ ]:
finalxdata.info()

# Model testing on unseen data and checking accuracy

In [ ]:
predy1 = tree_model.predict(finalxdata)
msetest1 = mean_squared_error(Ytestdata, predy1)
r_squaredtest1 = r2_score(Ytestdata, predy1)
rmse1 = np.sqrt(msetest1)
mae1 = mean_absolute_error(Ytestdata,predy1)
print("mean absolute Error:", mae1)
print("root Squared Error:", rmse1)
print("Mean Squared Error:", msetest1)
print("root Squared Error:", rmse1)
print("R-squared:", r_squaredtest1)


In [ ]:
predy2 =random_model.predict(finalxdata)
msetest2 = mean_squared_error(Ytestdata, predy2)
r_squaredtest2 = r2_score(Ytestdata, predy2)
rmse2 = np.sqrt(msetest2)
mae2 = mean_absolute_error(Ytestdata,predy2)
print("mean absolute Error:", mae2)
print("root Squared Error:", rmse2)

print("Mean Squared Error:", msetest2)
print("R-squared:", r_squaredtest2)

In [ ]:
predy3 = svr_model.predict(finalxdata)
msetest3 = mean_squared_error(Ytestdata, predy3)
r_squaredtest3 = r2_score(Ytestdata, predy3)
rmse3 = np.sqrt(msetest3)
mae3 = mean_absolute_error(Ytestdata,predy3)
print("mean absolute Error:", mae3)
print("root Squared Error:", rmse3)
print("Mean Squared Error:", msetest3)
print("R-squared:", r_squaredtest3)

In [ ]:
predy4 = gradient_model.predict(finalxdata)
msetest4 = mean_squared_error(Ytestdata, predy4)
r_squaredtest4 = r2_score(Ytestdata, predy4)
rmse4 = np.sqrt(msetest4)
mae4 = mean_absolute_error(Ytestdata,predy4)
print("mean absolute Error:", mae4)
print("root Squared Error:", rmse4)

print("Mean Squared Error:", msetest4)
print("R-squared:", r_squaredtest4)

In [ ]:
predy5 = gradient_model.predict(finalxdata)
msetest5 = mean_squared_error(Ytestdata, predy5)
r_squaredtest5 = r2_score(Ytestdata, predy5)
rmse5 = np.sqrt(msetest5)
mae5 = mean_absolute_error(Ytestdata,predy5)
print("mean absolute Error:", mae5)
print("root Squared Error:", rmse5)

print("Mean Squared Error:", msetest5)
print("R-squared:", r_squaredtest5)

In [ ]:
base_models = [("decision_tree",DecisionTreeRegressor()),
               ("SVR", SVR(kernel = "rbf")),
               ("random_forest", RandomForestRegressor(n_estimators = 150, random_state =42)),
               ('Gradient_model',GradientBoostingRegressor(n_estimators = 150, learning_rate = 0.1, max_depth =1, random_state =42)),
               ("Nu_svr", NuSVR(kernel ="rbf",nu = 0.5))
               ]

# Ensemble learning model testing on unseen data

1st ensemble learning model

In [ ]:
predy6 = stacking_model.predict(finalxdata)
msetest6 = mean_squared_error(Ytestdata, predy6)
r_squaredtest6 = r2_score(Ytestdata, predy6)

rmse6 = np.sqrt(msetest6)
mae6 = mean_absolute_error(Ytestdata,predy6)
print("mean absolute Error:", mae6)
print("root Squared Error:", rmse6)

print("Mean Squared Error:", msetest6)
print("R-squared:", r_squaredtest6)

2nd ensemble learning model

In [ ]:

predy7 = voting_model.predict(finalxdata)
msetest7 = mean_squared_error(Ytestdata, predy7)
r_squaredtest7 = r2_score(Ytestdata, predy7)
rmse7 = np.sqrt(msetest7)
mae7 = mean_absolute_error(Ytestdata,predy7)
print("mean absolute Error:", mae7)
print("root Squared Error:", rmse7)

print("Mean Squared Error:", msetest7)
print("R-squared:", r_squaredtest7)

4th ensemble learning model

In [ ]:
predy8 = bagging_model.predict(finalxdata)
msetest8 = mean_squared_error(Ytestdata, predy8)
r_squaredtest8 = r2_score(Ytestdata, predy8)
rmse8 = np.sqrt(msetest8)
mae8 = mean_absolute_error(Ytestdata,predy8)
print("mean absolute Error:", mae8)
print("root Squared Error:", rmse8)
print("Mean Squared Error:", msetest8)
print("R-squared:", r_squaredtest8)

# Final model that will be deployed.
* We used closs validation to find the best paramters

In [ ]:
finalPred = voting_final.predict(finalxdata)
msetestfinal = mean_squared_error(Ytestdata, finalPred)
r_squaredtestfinal = r2_score(Ytestdata,finalPred)
rmsefinal = np.sqrt(msetestfinal)
maefinal = mean_absolute_error(Ytestdata,finalPred)
print("mean absolute Error:", maefinal)
print("root Squared Error:", rmsefinal)
print("Mean Squared Error:", msetestfinal)
print("R-squared:", r_squaredtestfinal)

In [ ]:
columns = finalxdata.columns

In [ ]:
columns

#Testing the model on small input

In [ ]:
voting_final.predict(finalxdata[1000:1050])

In [ ]:
Ytestdata[1000:1050]

In [ ]:
Xtestdata


In [ ]:
Ytestdata

# Packaging and downloading the model for development

In [ ]:
from joblib import dump
import joblib

In [ ]:
dump(scaler,'drive/MyDrive/IntroAi/scaler2.joblib' )

In [ ]:
dump(voting_final, 'drive/MyDrive/IntroAi/voting.joblib')

In [ ]:
model = joblib.load('drive/MyDrive/IntroAi/voting.joblib')

In [ ]:
scaler2 = joblib.load('drive/MyDrive/IntroAi/scaler2.joblib')

In [ ]:
encoded = scaler2.transform(Xtestdata)

In [ ]:
model.predict(encoded)

# Finding of the confidence score of the our final model for deployment

In [ ]:
base_model_predictions = []
for model in voting_final.estimators_:
    base_model_predictions.append(model.predict(finalxdata))

variance_of_predictions = np.var(base_model_predictions)

confidence_score = 1 - (msetestfinal / variance_of_predictions)

print("Confidence Score:", confidence_score)